In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
X_train = np.load('Data/Train/X.npy')
X_prime_train = np.load('Data/Train/X_prime.npy')
feature_info_train = np.load('Data/Train/feature_information.npy')

In [3]:
X_test = np.load('Data/Test/X.npy')
X_prime_test = np.load('Data/Test/X_prime.npy')
feature_info_test = np.load('Data/Test/feature_information.npy')

In [4]:
isFeatureReal_train = pd.DataFrame(feature_info_train)
isFeatureReal_test = pd.DataFrame(feature_info_test)

In [5]:
isFeatureReal_test.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,0,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [6]:
isFeatureReal_train.head(10)

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,0
5,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,1
7,0,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,1,1,0,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [7]:
def least_square_regression(X, Y):
    m, k = X.shape[0], X.shape[1]
    z = np.ones((m,1))
    X = np.append(X, z, axis=1)
    mat = np.linalg.inv(np.dot(np.transpose(X), X))
    w = np.dot(mat, np.dot(np.transpose(X), Y))
    return w

In [8]:
w = least_square_regression(X_prime_train, X_train)

In [9]:
w.shape

(107, 106)

In [10]:
w[0,-1]

-1.6952802009417844e-14

In [11]:
def predict(w, isFeatureReal, X_prime):
    X_prime = pd.DataFrame(X_prime)
    num_features = w.shape[0]
    for k in range(num_features-1):
        col_to_predict = k
        w_k = w[k]
        new_w = np.delete(w_k, col_to_predict)
        for idx, _ in isFeatureReal.loc[isFeatureReal[col_to_predict] == 0].iterrows():
            row = X_prime.iloc[idx]
            new_row = row.drop(col_to_predict)
            pred = - (np.dot(new_w, new_row) + w_k[-1])/w_k[col_to_predict]
            X_prime[col_to_predict].iloc[idx] = pred    
    return X_prime

In [12]:
X_pred_train = predict(w, isFeatureReal_train, X_prime_train)
X_pred_train.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,3.013699e-01,-2.244966e-12,8.000000e-01,0.02174,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000e+00,0.0
1,4.520548e-01,4.727738e-02,8.000000e-01,0.00000,0.0,0.122449,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000e+00,0.0
2,2.876712e-01,1.372439e-01,-8.350452e-15,0.00000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000e+00,0.0
3,2.990791e-13,1.502118e-01,4.000000e-01,0.00000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000e+00,0.0
4,1.848534e-13,2.207030e-01,8.000000e-01,0.00000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.284231e-14,-1.0


In [13]:
def calculateError(X_prime, X):
    predictedData = X_prime.to_numpy()
    trueData = X

    diff = (trueData - predictedData)**2
    sumDiff = np.sum(diff, axis = 1)
#     err = np.mean(np.sqrt(sumDiff))
    err = np.sqrt(np.mean(sumDiff))

    return err

In [14]:
errTrain = calculateError(X_pred_train, X_train)
print("Training Error:", errTrain)

Training Error: 0.3713595158154271


In [15]:
X_pred_test = predict(w, isFeatureReal_test, X_prime_test)
X_pred_test.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,1.095890e-01,0.144430,4.000000e-01,0.000000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2.876712e-01,0.051677,5.333333e-01,0.000000,0.0,0.500000,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1.506849e-01,0.219011,7.333333e-01,0.000000,0.0,0.397959,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,3.698630e-01,0.099418,-1.198755e-15,0.076881,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,3.060673e-13,0.125398,-5.352378e-15,0.000000,0.0,0.295918,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [16]:
errTest = calculateError(X_pred_test, X_test)
print("Training Error:", errTest)

Training Error: 0.36347680846333613


Ridge regression:

In [27]:
def ridge_regression(X, Y, l = 0.01):
    m, k = X.shape[0], X.shape[1]
    z = np.ones((m,1))
    X = np.append(X, z, axis=1)
    mat = np.linalg.inv(np.dot(np.transpose(X), X) + l*np.identity(k+1))
    w = np.dot(mat, np.dot(np.transpose(X), Y))
    return w

In [35]:
w_ridge_regression = ridge_regression(X_prime_train, X_train, l = 0.01)
print(w_ridge_regression.shape)

(107, 106)


In [37]:
X_pred_train_ridge = predict(w_ridge_regression, isFeatureReal_train, X_prime_train)
X_pred_train_ridge.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,0.301370,0.002465,0.80000,0.02174,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.000000
1,0.452055,0.047277,0.80000,0.00000,0.0,0.122449,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.000000
2,0.287671,0.137244,0.00239,0.00000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.000000
3,0.001667,0.150212,0.40000,0.00000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.000000
4,0.003718,0.220703,0.80000,0.00000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005287,-1.001446


In [38]:
errTrainRidge = calculateError(X_pred_train_ridge, X_train)
print("Training Error:", errTrainRidge)

Training Error: 0.43755291756234876


In [39]:
X_pred_test_ridge = predict(w_ridge_regression, isFeatureReal_test, X_prime_test)
X_pred_test_ridge.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,0.109589,0.144430,0.400000,0.000000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.287671,0.051677,0.533333,0.000000,0.0,0.500000,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.150685,0.219011,0.733333,0.000000,0.0,0.397959,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.369863,0.099418,0.000237,0.076881,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-0.000129,0.125398,0.002137,0.000000,0.0,0.295918,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [40]:
errTestRidge = calculateError(X_pred_test_ridge, X_test)
print("Training Error:", errTestRidge)

Training Error: 0.4328035565246928
